<div class="alert alert-info">

**Note:**    WARNING: This notebook uses a new, early release Chameleon python API. The notebook and API will likely be updated without notice. To request a new python feature, please create a help desk ticket. 
    

</div>

## Stitching Networks between Chameleon and an ExoGENI Stitchport

An example that can be used to stitch to an ExoGENI Stitchport

#### Modules 

- [Reserve Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)
- [Get Network by Name](../modules-python/network/get_network_by_name.ipynb)

TODO: add the rest of the module links when they are ready

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [4]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

#Config with your project
#chi.set('project_name', 'ExoGENI@Chameleon') # Replace with your project name
chi.set('project_name', 'CH-822154')

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

#GENI Pem file
geni_pem_file='/home/pruth/work/geni-mwhicks01.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
stitched_network_name = network_name+"Stitched"
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name_network = username+'LeaseNet'
lease_name_servers = username+'LeaseServers'
lease_name_stitch = username+'LeaseStitch'


#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=2

#Network Config
cidr='192.168.42.0/24'

#optionally set the OpenFlow controller (set ot None for non-OpenFlow swtich)
#of_controller_ip=None
of_controller_ip="192.5.87.215"
of_controller_port="6653"


#Set the name of the VFC
vswitch_name=username+"SDN"

#Stitchport URL
stitchport_url='http://geni-orca.renci.org/owl/ion.rdf#AL2S/Chameleon/Cisco/6509/GigabitEthernet/1/2'
stitchport_vlan='3280'

## Create the Network (OpenFlow Optional)

#### Create a Lease at Chicago

In [6]:
#Set the region
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)


# Build list of reservations (in this case there is only one reservation)
reservation_list = []

if of_controller_ip:
    print("OpenFlow")
    create_network(network_name, of_controller_ip=of_controller_ip, of_controller_port=of_controller_port, vswitch_name=vswitch_name,
                provider='physnet1', port_security_enabled=True)
else:
    print("No OpenFlow")
    create_network(network_name, vswitch_name=vswitch_name,
                provider='physnet1', port_security_enabled=True)

In [8]:
#Get the network by name
network = get_network(network_name)
    
#Print the lease info
print(json.dumps(network, indent=2))

{
  "provider:physical_network": "physnet1",
  "ipv6_address_scope": null,
  "revision_number": 1,
  "port_security_enabled": true,
  "provider:network_type": "vlan",
  "id": "16796f29-6b8d-44e9-8861-27bdd1459962",
  "router:external": false,
  "availability_zone_hints": [],
  "availability_zones": [],
  "ipv4_address_scope": null,
  "shared": false,
  "project_id": "ae76673270164b048b59d3bd30676721",
  "status": "ACTIVE",
  "subnets": [],
  "description": "OFController=192.5.87.215:6653,VSwitchName=mwhicks2SDN",
  "tags": [],
  "updated_at": "2021-03-17T20:29:06Z",
  "provider:segmentation_id": 3266,
  "name": "mwhicks2Net",
  "admin_state_up": true,
  "tenant_id": "ae76673270164b048b59d3bd30676721",
  "created_at": "2021-03-17T20:29:06Z",
  "mtu": 1500
}


#### Get the Nework Reservation

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [13]:
#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
network_vlan = network['provider:segmentation_id']
print('network_vlan: ' + str(network_vlan))

Network ID: 16796f29-6b8d-44e9-8861-27bdd1459962
network_vlan: 3266


#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [17]:
subnet = create_subnet(subnet_name, network_id, cidr=cidr, gateway_ip=None)

print(json.dumps(subnet, indent=2))

{
  "service_types": [],
  "description": "",
  "enable_dhcp": true,
  "tags": [],
  "network_id": "16796f29-6b8d-44e9-8861-27bdd1459962",
  "tenant_id": "ae76673270164b048b59d3bd30676721",
  "created_at": "2021-03-17T20:34:55Z",
  "dns_nameservers": [],
  "updated_at": "2021-03-17T20:34:55Z",
  "gateway_ip": "192.168.42.1",
  "ipv6_ra_mode": null,
  "allocation_pools": [
    {
      "start": "192.168.42.2",
      "end": "192.168.42.254"
    }
  ],
  "host_routes": [],
  "revision_number": 0,
  "ip_version": 4,
  "ipv6_address_mode": null,
  "cidr": "192.168.42.0/24",
  "project_id": "ae76673270164b048b59d3bd30676721",
  "id": "e287ca82-cf78-4aa8-96a4-d301de7f39c0",
  "subnetpool_id": null,
  "name": "mwhicks2subnet"
}


#### Add a Router

TODO: add links here

In [18]:
router = create_router(router_name, gw_network_name='public')

print(json.dumps(router, indent=2))

{
  "status": "ACTIVE",
  "external_gateway_info": {
    "network_id": "44b38c44-2a42-4b6d-b129-6c8f1b2a1375",
    "enable_snat": true,
    "external_fixed_ips": [
      {
        "subnet_id": "c3950603-9e04-4cc5-be8d-1efbfe59fc0a",
        "ip_address": "192.5.87.37"
      }
    ]
  },
  "availability_zone_hints": [],
  "availability_zones": [],
  "description": "",
  "tags": [],
  "tenant_id": "ae76673270164b048b59d3bd30676721",
  "created_at": "2021-03-17T20:35:20Z",
  "admin_state_up": true,
  "updated_at": "2021-03-17T20:35:21Z",
  "flavor_id": null,
  "revision_number": 3,
  "routes": [],
  "project_id": "ae76673270164b048b59d3bd30676721",
  "id": "6dea426f-6e24-461b-bde3-84641e791108",
  "name": "mwhicks2Router"
}


#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
add_subnet_to_router_by_name(router_name, subnet_name)

#### Add a Circuit Stitched to ExoGENI

In [20]:
chi.set('project_name', 'ExoGENI@Chameleon')

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
create_network(stitched_network_name, vswitch_name=vswitch_name,
                provider='exogeni', port_security_enabled=True)

#Get the lease by name
stitched_network = get_network(stitched_network_name)
    
#Print the lease info
print(json.dumps(stitched_network, indent=2))

Unauthorized: The request you have made requires authentication. (HTTP 401) (Request-ID: req-0d2c4462-abd0-4f55-ab31-977adc7f8c56)

#### Get the Stitched Network Reservation and VLAN

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
stitch_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', stitch_lease['reservations']))[0]['id']

print("stitch_reservation_id: " + stitch_reservation_id)

#Get the network
network = get_network(stitched_network_name)
#print(json.dumps(network, indent=2))

#Get the network ID
network_id = network['id']
print('Network ID: ' + str(network_id))

#Get the VLAN tag (needed for ExoGENI stitching)
exogeni_vlan = network['provider:segmentation_id']
print('exogeni_vlan: ' + str(exogeni_vlan))



## Create the Servers

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name_servers, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
server_lease = get_lease(lease_name_servers)
    
#Print the lease info
print(json.dumps(server_lease, indent=2))

In [ ]:
compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', server_lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', server_lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Start the Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)

#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit


In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env chameleon_vlan="$exogeni_vlan" stitchport_url="$stitchport_url" stitchport_vlan="$stitchport_vlan" geni_pem="$geni_pem_file" bash

echo 'chameleon_vlan ' $chameleon_vlan ', stitchport_url ' $stitchport_url ', stitchport_vlan ' $stitchport_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $chameleon_vlan -sp2 $stitchport_url -vlan2 $stitchport_vlan -c $geni_pem

### Delete Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(get_router_id(router_name), get_subnet_id(subnet_name))

In [ ]:
delete_router(get_router_id(router_name))

In [ ]:
delete_subnet(get_subnet_id(subnet_name))

In [ ]:
delete_network(get_network_id(network_name))

#### Release Leases

In [ ]:
delete_lease(lease_name_network)
delete_lease(lease_name_servers)
delete_lease(lease_name_stitch)